In [91]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm import tqdm

In [99]:
def loadDataframe(soup, df, position):
    job_dict = {}
    for jobs in soup.find_all('div', {'data-feat':"searched_jobs"}):
        job_dict['title'] = jobs.find('span', {'class':'iDyMWN'}).get_text()
        job_dict['date_posted'] = jobs.find('span', {'class':'eEFkdA'}).get_text().split(':')[1].strip()
        job_dict['location'] = jobs.find('p', {'class':'cMsiEU'}).get_text()
        job_dict['company'] = jobs.find('div', {'class':'gNMZoz'}).get_text()
        job_dict['category'] = position
        job_dict['url'] = f"https://www.jobs.ch{jobs.find('a', {'class':'zoplL'}).get('href')}"
        # print(job_dict['url'])

        job_df = pd.DataFrame(job_dict, index=[0])

        df = pd.concat([df, job_df], ignore_index=True)

        job_dict = {}
    return(df)

In [89]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [104]:
# https://www.jobs.ch/en/vacancies/?term=data%20scientist
# https://www.jobs.ch/en/vacancies/?page=2&term=data%20scientist

positions = [
    "Data Scientist",
    "Data Analyst",
    "Python Developer",
    "Data Engineer",
    "Data Manager",
    "Data Architect",
    "Big Data Analyst",
    "Data Python"
]

df = pd.DataFrame()


for position in tqdm(positions, desc=f"Running position: {position}"):
    r = requests.get('https://www.jobs.ch/en/vacancies', params={'term':position})
    soup = BeautifulSoup(r.text, 'html.parser')
    page_numbers = int(soup.find('span', {'class':'Span-sc-1ybanni-0 Text__span-sc-1lu7urs-12 Text-sc-1lu7urs-13 cUwwWd'}).get_text().split('/')[1].strip())
    df = loadDataframe(soup, df, position)
    # df
    for page in range(2, page_numbers+1):
        payload = {'page':page, 'term':position}
        r = requests.get('https://www.jobs.ch/en/vacancies', params=payload)
        if(r.status_code == 200):
            # print('current url: ', r.url, end='\r')
            soup = BeautifulSoup(r.text, 'html.parser')
            df = loadDataframe(soup, df, position)
        else:
            print('Error reading webpage', r.url, ' with error status code: ', r.status_code)
        print('Size of Data Frame: ', len(df), 'current url: ', r.url, end='\r')
df.tail(10)

Running position: Data Scientist:   0%|                                                           | 0/8 [00:00<?, ?it/s]

Running position: Data Scientist:  12%|██████▍                                            | 1/8 [00:22<02:40, 22.91s/it]

Running position: Data Scientist:  25%|████████████▊                                      | 2/8 [00:48<02:28, 24.73s/it]

Running position: Data Scientist:  38%|███████████████████▏                               | 3/8 [00:55<01:21, 16.24s/it]

Running position: Data Scientist:  50%|█████████████████████████▌                         | 4/8 [02:04<02:29, 37.38s/it]

Running position: Data Scientist:  62%|███████████████████████████████▉                   | 5/8 [03:41<02:56, 58.68s/it]

Running position: Data Scientist:  75%|██████████████████████████████████████▎            | 6/8 [03:56<01:27, 43.84s/it]

Running position: Data Scientist:  88%|████████████████████████████████████████████▋      | 7/8 [04:08<00:33, 33.51s/it]

Running position: Data Scientist: 100%|███████████████████████████████████████████████████| 8/8 [04:40<00:00, 35.08s/it]

,title,date_posted,location,company,category,url
5042,Ph.D. student position in laser ablation (LA),14 November 2023,Basel,Universität Basel,Data Python,https://www.jobs.ch/en/vacancies/detail/48ec0787-f4ab-4834-92ac-f23dfa0fb456/?source=vacancy_search
5043,Senior DevOps Engineer,18 August 2023,Zürich Oerlikon,PDF Tools AG,Data Python,https://www.jobs.ch/en/vacancies/detail/53aae677-7ba4-481f-905d-b02f8001f4b8/?source=vacancy_search
5044,Digital Manufacturing Consultant Industry X (all genders),07 November 2023,Basel|Bern|Geneve|Lugano|Zurich,Accenture,Data Python,https://www.jobs.ch/en/vacancies/detail/f989f47a-7577-42a0-a14e-eca381a75994/?source=vacancy_search
5045,Senior Consultant - FS Treasury and Liquidity Risk Consulting,14 November 2023,Zürich,KPMG,Data Python,https://www.jobs.ch/en/vacancies/detail/3c78d516-b709-46e0-b52e-d0332b29ed28/?source=vacancy_search
5046,"2 PhD positions: Real structure of materials with local lattice instabilities 100%, Zurich, fixed-term",25 October 2023,Zurich,ETH Zürich,Data Python,https://www.jobs.ch/en/vacancies/detail/9c6a15eb-27ec-429f-ac10-2e80a8ddb703/?source=vacancy_search
5047,Senior DevOps Engineer,07 November 2023,Zürich,,Data Python,https://www.jobs.ch/en/vacancies/detail/f7e25b6a-9bd6-4e63-96cd-c8b68d66edf8/?source=vacancy_search
5048,Un poste d'assistant-e doctorant-e à 100%,12 October 2023,Genève,Université de Genève,Data Python,https://www.jobs.ch/en/vacancies/detail/7b296b13-1faa-4e56-9c5c-7f8519c34193/?source=vacancy_search
5049,Senior Consultant - FS Treasury and Liquidity Risk Consulting,15 November 2023,Zürich,KPMG,Data Python,https://www.jobs.ch/en/vacancies/detail/baf10d37-6838-4872-8326-8e79073c8e9b/?source=vacancy_search
5050,Revolutionize enterprise search with Generative AI - Internship 6 months COO_IT,31 October 2023,Geneva,,Data Python,https://www.jobs.ch/en/vacancies/detail/0f2c58ea-5326-41c2-ab3a-236bdbce3324/?source=vacancy_search
5051,Administrateur de base de données de production/Administratrice de base de données de production H/F,14 November 2023,Lausanne,CA Indosuez (Switzerland) SA,Data Python,https://www.jobs.ch/en/vacancies/detail/a399114f-af72-4f9a-815e-43d5847feeb6/?source=vacancy_search


# Number of Jobs that are Duplicate

In [114]:
len(df) - len(df.drop_duplicates(['title','date_posted','location','company']))

3484